### Oracle AI Vector Search wrapped as a llama-index custom Vector Store

* inspired by: https://docs.llamaindex.ai/en/stable/examples/low_level/vector_store.html
* updated for **Llamaindex 0.10+**

In this **first demo** we show:
* how to embed a Text using OCI GenAI Embeddings (Cohere V3)
* How to query the Oracle AI Vector Store
* How to create a simplified QA retriever using LlamaIndex
* How to answer the question using a chatbot based on a LLM

In [1]:
import logging


from llama_index.core.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

import ads
from oci_utils import load_oci_config
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
from oracle_vector_db import OracleVectorStore

from prepare_chain_4_chat import create_chat_engine

from config import EMBED_MODEL, TOP_K, TOP_N

from config_private import COMPARTMENT_OCID

/Users/lsaetta/miniforge3/envs/rag_env03/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
print(ads.__version__)

2.11.3


In [2]:
# disable tracing
ADD_PHX_TRACING = False

# this is the endpoint after GA, for now Chicago Region
ENDPOINT = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"

In [3]:
# Configure logging
logger = logging.getLogger("ConsoleLogger")

# to avoid duplication of logging
if not logger.handlers:
    logger.setLevel(logging.INFO)

    handler = logging.StreamHandler()
    handler.setLevel(logging.INFO)

    formatter = logging.Formatter("%(asctime)s - %(message)s")
    handler.setFormatter(formatter)
    logger.addHandler(handler)

In [4]:
# setup for security
# in this example you need to have your api key in $HOME/.oci
# inside Data Science you can setup Resource Principal
oci_config = load_oci_config()

api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual

embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=EMBED_MODEL,
    auth=api_keys_config,
    truncate="END",
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

# instantiate the client for the LLM
llm_oci = GenerativeAI(
    compartment_id=COMPARTMENT_OCID,
    # you can get these params also from config
    max_tokens=1024,
    temperature=0.1,
    client_kwargs={"service_endpoint": ENDPOINT},
)

#### Using the wrapper for AI Vector Sesarch to find documents relevant to the query

In [5]:
v_store = OracleVectorStore(verbose=True)

In [6]:
question = (
    # "What is JSON Relational Duality in Oracle Database 23c? Explain with details"
    "What is Oracle Data Guard? Can it be used for Disaster Recovery"
)

In [7]:
# embed the query using OCI GenAI

query_embedding = embed_model.embed_documents([question])[0]

#  wrap in llama-index
query_obj = VectorStoreQuery(query_embedding=query_embedding, similarity_top_k=TOP_K)

In [8]:
%%time

q_result = v_store.query(query_obj)

2024-03-30 12:26:01,516 - SQL Query: select V.id, C.CHUNK, C.PAGE_NUM, 
                            VECTOR_DISTANCE(V.VEC, :1, COSINE) as d,
                            B.NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH  FIRST 8 ROWS ONLY
2024-03-30 12:26:01,516 INFO ConsoleLogger: SQL Query: select V.id, C.CHUNK, C.PAGE_NUM, 
                            VECTOR_DISTANCE(V.VEC, :1, COSINE) as d,
                            B.NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH  FIRST 8 ROWS ONLY
2024-03-30 12:26:01,936 - Query duration: 0.6 sec.
2024-03-30 12:26:01,936 INFO ConsoleLogger: Query duration: 0.6 sec.


CPU times: user 21.4 ms, sys: 5.17 ms, total: 26.6 ms
Wall time: 638 ms


#### Displays results from AI Vector Search

In [9]:
for n, id, sim in zip(q_result.nodes, q_result.ids, q_result.similarities):

    print(n.text)
    print("")
    print(n.metadata)
    print(f"Similarity: {round(sim, 4)}")
    print("-------------------------------")

7 High Availability Data Guard Oracle Data Guard Redo Decryption for Hybrid Disaster Recovery Configurations Oracle Data Guard now provides the capability to decrypt redo operations in hybrid cloud disaster recovery configurations where the cloud database is encrypted with Transparent Data Encryption (TDE) and the on-premises database is not. Hybrid disaster recovery (DR) with Data Guard is now more flexible and easy to configure. Hybrid disaster recovery for the Oracle Database allows you to expand outage and data protection to take advantage of the automation and resources of Oracle Cloud Infrastructure (OCI). By enabling the ability to quickly configure disaster recovery in OCI, even in cases where on-premises databases might not already be encrypted with Transparent Data Encryption (TDE), the steps required to configure hybrid disaster recovery environments and prepare on-premises databases for a DR configuration with cloud databases in OCI have been greatly reduced. Related Resour

#### Integrate in the RAG picture (the entire chain: Embeddings -> Vector Store -> LLM)

In [10]:
# replaced ServiceContext with Settings (llamaindex > 0.10)
Settings.embed_model = embed_model
Settings.llm = llm_oci

index = VectorStoreIndex.from_vector_store(vector_store=v_store)

query_engine = index.as_query_engine(similarity_top_k=TOP_K)

In [11]:
%%time

response = query_engine.query(question)

print(f"Question: {question}")
print("")
print(response.response)
print("")

2024-03-30 12:26:51,960 - SQL Query: select V.id, C.CHUNK, C.PAGE_NUM, 
                            VECTOR_DISTANCE(V.VEC, :1, COSINE) as d,
                            B.NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH  FIRST 8 ROWS ONLY
2024-03-30 12:26:51,960 INFO ConsoleLogger: SQL Query: select V.id, C.CHUNK, C.PAGE_NUM, 
                            VECTOR_DISTANCE(V.VEC, :1, COSINE) as d,
                            B.NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH  FIRST 8 ROWS ONLY
2024-03-30 12:26:52,391 - Query duration: 0.6 sec.
2024-03-30 12:26:52,391 INFO ConsoleLogger: Query duration: 0.6 sec.


Question: What is Oracle Data Guard? Can it be used for Disaster Recovery

Oracle Data Guard is a tool that can be used to replicate, manage and protect databases in an Oracle environment. It ensures data is continuously available and protects against data loss. With Oracle Data Guard, you can use hybrid disaster recovery to quickly configure disaster recovery in Oracle Cloud Infrastructure (OCI), even if the on-premises databases are not encrypted with Transparent Data Encryption (TDE). This makes it easy to configure hybrid disaster recovery environments and prepare on-premises databases for a DR configuration with cloud databases in OCI. 

Would you like to know more about Oracle Data Guard or Disaster Recovery? 

CPU times: user 216 ms, sys: 97.8 ms, total: 314 ms
Wall time: 7.36 s


#### Using the chat engine: a single line of code

In [12]:
chat_engine, token_counter = create_chat_engine(token_counter=None, verbose=False)

2024-03-30 12:26:58,842 - Calling create_chat_engine()...
2024-03-30 12:26:58,842 INFO ConsoleLogger: Calling create_chat_engine()...
2024-03-30 12:26:58,846 - ------------------------
2024-03-30 12:26:58,846 INFO ConsoleLogger: ------------------------
2024-03-30 12:26:58,846 - Configuration used:
2024-03-30 12:26:58,846 INFO ConsoleLogger: Configuration used:
2024-03-30 12:26:58,848 - OCI cohere.embed-multilingual-v3.0 for embeddings...
2024-03-30 12:26:58,848 INFO ConsoleLogger: OCI cohere.embed-multilingual-v3.0 for embeddings...
2024-03-30 12:26:58,850 - Using Oracle AI Vector Search...
2024-03-30 12:26:58,850 INFO ConsoleLogger: Using Oracle AI Vector Search...
2024-03-30 12:26:58,852 - Using COHERE as LLM...
2024-03-30 12:26:58,852 INFO ConsoleLogger: Using COHERE as LLM...
2024-03-30 12:26:58,853 - Retrieval parameters:
2024-03-30 12:26:58,853 INFO ConsoleLogger: Retrieval parameters:
2024-03-30 12:26:58,854 - TOP_K: 8
2024-03-30 12:26:58,854 INFO ConsoleLogger: TOP_K: 8
2024-0

In [13]:
response = chat_engine.chat(question)

In [14]:
print(response)

Oracle Data Guard is a high availability technology that helps protect against planned and unplanned downtimes. It enables the creation of one or more standby databases that can reside at the same site, at a remote site, or in the cloud. These standby databases continuously receive and apply changes from the primary database, ensuring they are up-to-date and synchronized.

Data Guard's key role is in disaster recovery, providing a robust mechanism to recover from various types of disasters, such as:

1. Site Disasters: Data Guard can switch to a standby database at a remote site in the event of a natural disaster, such as a fire or flood affecting the primary site. This ensures continuity with minimal data loss.

2. Database Corruption or Hardware Failure: If the primary database experiences corruption or hardware issues, Data Guard can quickly switch to a standby database, minimizing outage times while the primary database is being repaired.

3. Human Error: Accidental data loss or in

In [15]:
# if you want to count the token
token_counter.completion_llm_token_count

343

#### The chain keeps the conversation

In [16]:
question2 = "Can it be used to implement RPO close to zero?"

response2 = chat_engine.chat(question2)

In [17]:
print(response2)

Yes, Oracle Data Guard can help achieve a Recovery Point Objective (RPO) that is close to zero. This means that in the event of a disaster, the data loss would be minimal, virtually approaching zero.

Data Guard achieves this by continuously replicating and applying changes from the primary database to the standby databases in real-time or near-real time. The synchronous mode of Data Guard, known as Maximum Protection, ensures that the standby database remains up-to-date with the primary database. This real-time synchronization significantly reduces the time window for data loss, thereby minimizing the RPO.

With Data Guard, the standby database can be configured to receive and apply changes as they occur on the primary database. This continuous replication means that when a disaster occurs, the standby database can quickly take over with nearly all transactions accounted for, resulting in an RPO that's close to zero.

The actual RPO achieved depends on several factors, including the n